In [13]:
import joblib
import os
import numpy as np
from tqdm import tqdm 
import torch
# from keras.backend.tensorflow_backend import set_session ## 和tf.keras不同

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
#0 是默认值，输出所有信息
# 1 是屏蔽通知信息
# 2 是屏蔽通知和警告信息
###################################
###########loading dataset#########
###################################
l_N = 60   # lengthscale对的数量
k_ ='rbf'
nx = 32
ny = 32
num_samples = 100   # 每个随机场采样的样本数


In [3]:

datadir = "/home/huangjg/MyFiles/deep-uq-paper/data"

cellcentersFile = "cellcenters_"+"nx_"+str(nx)+"_"+str(ny)+".pkl"
cellcenters = joblib.load(os.path.join(datadir,cellcentersFile))
AllDataFile = k_+"_l_N_"+str(l_N)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+".pkl"

Train_all_data = joblib.load(os.path.join(datadir,AllDataFile))
print("数据集加载成功！")

数据集加载成功！


In [9]:

Input = []
Output =[]
for i in range(l_N):
    for j in range(num_samples):
        randomFiled_input = Train_all_data[i]['randomfield_sample'][j].reshape((1,-1))
        outputs =Train_all_data[i]['outputs'][j].reshape((1,-1))
        for k in range(nx*ny):
            input = np.append(randomFiled_input,cellcenters[k])
            Input.append(input)
            Output.append(outputs[0,k])

Inputs = np.array(Input)
Input_Dim = Inputs.shape[1]
labels = np.array(Output)
print("训练样本有",labels.shape[0])

# train_mean_label = np.mean(labels,axis=0)
# train_std_label = np.std(labels,axis=0)
# print(train_mean_label )
# print(train_std_label)
 

训练样本有 6144000


In [8]:
# joblib.dump([train_mean_label,train_std_label],filename="norm_std.pkl")
train_mean_label,train_std_label = joblib.load(filename="norm_std.pkl")


In [10]:
nrom_labels_train =  (labels-train_mean_label)/train_std_label
print(nrom_labels_train.shape)
TrainData = {"Inputs":Inputs,
            "labels":nrom_labels_train,
            "normal":[train_mean_label,train_std_label],}
TrainDataFile = k_+"_l_N_"+str(l_N)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+"_train.pkl"

joblib.dump(TrainData,filename=os.path.join(datadir,TrainDataFile))


(6144000,)


['/home/huangjg/MyFiles/deep-uq-paper/data/rbf_l_N_60_num_samples_100_nx_32_ny_32_train.pkl']

: 

# 提取lengthscale  作为训练任务标志

In [6]:
# 提取出length scale

lengthscale_pair_array = []
for i in range(l_N):
    lengthscale_pair_array.append(Train_all_data[i]['lengthscale_pair'])

lengthscale_pair_array = np.array(lengthscale_pair_array)
print("训练样本有",lengthscale_pair_array.shape)
joblib.dump(lengthscale_pair_array,filename=os.path.join(datadir,"lengthscale_pair_array.pkl"))

训练样本有 (60, 2)


['/home/huangjg/MyFiles/deep-uq-paper/data/lengthscale_pair_array.pkl']

In [10]:
lengthscale_pair_array

array([[0.04031616, 0.51471225],
       [0.4079415 , 0.23681085],
       [0.06788817, 0.80069876],
       [0.96598086, 0.11009247],
       [0.31877116, 0.38358789],
       [0.10932374, 0.90150841],
       [0.78900085, 0.06676085],
       [0.2290775 , 0.05234551],
       [0.98914405, 0.12080711],
       [0.66284873, 0.78377407],
       [0.71095387, 0.37833149],
       [0.87545583, 0.63759772],
       [0.6962953 , 0.05039404],
       [0.05579499, 0.46613041],
       [0.78154703, 0.30716332],
       [0.64799809, 0.77843295],
       [0.75602611, 0.19179143],
       [0.40386264, 0.27449089],
       [0.61259102, 0.18466561],
       [0.3849052 , 0.87286271],
       [0.49678404, 0.27724455],
       [0.48417573, 0.34920035],
       [0.11754271, 0.27571643],
       [0.49536241, 0.98122137],
       [0.44409481, 0.10900082],
       [0.73942936, 0.29883236],
       [0.10329725, 0.20667263],
       [0.55161422, 0.79806521],
       [0.61012774, 0.264826  ],
       [0.95364232, 0.51983881],
       [0.

In [ ]:
AllDataFile = k_+"_l_N_"+str(100)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+"_val.pkl"

Val_all_data = joblib.load(os.path.join(datadir,AllDataFile))
print("数据集加载成功！例子数量为",str(len(Val_all_data)))
InputVal = []
OutputVal =[]
for i in tqdm(range(50)):
    for j in range(num_samples):
        randomFiled_input = Val_all_data[i]['randomfield_sample'][j].reshape((1,-1))
        outputs =Val_all_data[i]['outputs'][j].reshape((1,-1))
        for k in range(nx*ny):
            input = np.append(randomFiled_input,cellcenters[k])
            InputVal.append(input)
            OutputVal.append(outputs[0,k])

Inputs_Val = np.array(InputVal)
labels_val = np.array(OutputVal)
print("训练样本有",labels_val.shape[0])
norm_labels_val  = (labels_val-train_mean_label)/train_std_label
print(norm_labels_val.shape)
ValDtata=  {
            "normal":[train_mean_label,train_std_label],
            "Inputs_val":Inputs_Val,
            "labels_val":norm_labels_val}


ValDataFile = k_+"_l_N_"+str(l_N)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+"_val_pre50.pkl"
joblib.dump(ValDtata,filename=os.path.join(datadir,ValDataFile))

# 构建元学习的数据集

In [9]:
train_mean_label,train_std_label = joblib.load(filename="norm_std.pkl")
print(train_mean_label,train_std_label)

0.501192575295927 0.30546128200005446


In [4]:
meta_Train_all_data = []
for i in tqdm(range(l_N)):
    Output = []
    Input = []
    for j in range(num_samples):
        randomFiled_input = Train_all_data[i]['randomfield_sample'][j].reshape((1,-1))
        outputs =Train_all_data[i]['outputs'][j].reshape((1,-1))
        outputs  =  (outputs-train_mean_label)/train_std_label
        for k in range(nx*ny):
            input = np.append(randomFiled_input,cellcenters[k])
            Input.append(input)
            Output.append(outputs[0,k])
    meta_task = {"input":Input,
                "labels":Output,
                "lengthscale_pair":Train_all_data[i]["lengthscale_pair"]}
    meta_Train_all_data.append(meta_task)

TrainDataFile = k_+"_l_N_"+str(l_N)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+"_train_meta.pkl"

joblib.dump(meta_Train_all_data,filename=os.path.join(datadir,TrainDataFile))

100%|██████████| 60/60 [00:43<00:00,  1.38it/s]


['/home/huangjg/MyFiles/deep-uq-paper/data/rbf_l_N_60_num_samples_100_nx_32_ny_32_train_meta.pkl']

: 

In [14]:
TrainDataFile = k_+"_l_N_"+str(l_N)+"_num_samples_"+str(num_samples)+\
                    "_nx_"+str(nx)+"_ny_"+str(ny)+"_train_meta.pkl"

lengthscale_array  =joblib.load(os.path.join(datadir,"lengthscale_pair_array.pkl"))
meta_Train_all_data = joblib.load(os.path.join(datadir,TrainDataFile))

In [15]:
larray = []
for i in range(60):
    larray.append(meta_Train_all_data[i]['lengthscale_pair'])

larray = np.array(larray)

In [19]:
for i in range(60):
    if(larray[i] !=lengthscale_array[i]).any():
        print(i)

# 构建高保真模型的数据集